# Fine tune model

https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(270M).ipynb#scrollTo=Mkq4RvEq7FQr

In [1]:
from unsloth import FastLanguageModel

instruct_model = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=instruct_model,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060 Laptop GPU. Num GPUs = 1. Max memory: 5.668 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2025.12.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


# Dataset

In [3]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

In [4]:
from pathlib import Path

from datasets import Dataset

from src.prompt_train import TRAIN_ASSISTANT_PROMPT_NER_RETRIEVAL
from src.type import SyntheticSampleAnnotated, SyntheticSamplesAnnotated

filepath_input_dataset = Path.cwd().parent / "data" / "synthetic_samples" / "annotated_synthetic_samples_train.json"
with filepath_input_dataset.open("r") as f:
    dataset_pyd = SyntheticSamplesAnnotated.model_validate_json(f.read())


def convert_sample_to_chatml(sample: SyntheticSampleAnnotated) -> list[dict[str, str]]:
    return [
        {"role": "system", "content": TRAIN_ASSISTANT_PROMPT_NER_RETRIEVAL},
        {"role": "user", "content": sample.sentence},
        {"role": "assistant", "content": sample.annotation.model_dump_json()},
    ]


dataset = Dataset.from_dict({"conversations": [convert_sample_to_chatml(sample) for sample in dataset_pyd.samples]})


def formatting_prompts_func(examples: dict) -> dict:
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False).removeprefix("<bos>")
        for convo in convos
    ]
    return {
        "text": texts,
    }


dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

# Train model

In [9]:
from transformers import DataCollatorForSeq2Seq
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    packing=False,  # Can make training 5x faster for short sequences.
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps=60,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/240 [00:00<?, ? examples/s]

In [10]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=16):   0%|          | 0/240 [00:00<?, ? examples/s]

In [11]:
import torch

# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060 Laptop GPU. Max memory = 5.668 GB.
1.203 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 240 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.048900
2,2.971800
3,3.174000
4,2.711300
5,2.106500
6,1.064900
7,0.623200
8,0.267000
9,0.179300
10,0.159700


# Save

In [15]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

In [16]:
# model.save_pretrained_gguf("model_llama_3_1b_it", tokenizer, quantization_method="q4_k_m")
model.save_pretrained_gguf("model", tokenizer,)

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /home/pdesj/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:30<00:00, 30.54s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:05<00:00,  5.40s/it]


Unsloth: Merge process complete. Saved to `/home/pdesj/work/github/ner-llm-finetuning/src/model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['Llama-3.2-1B-Instruct.BF16.gguf']
Unsloth: [2] Converting GGUF bf16 into q8_0. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['Llama-3.2-1B-Instruct.Q8_0.gguf']
Unsloth: example usage for text only LLMs: llama

{'save_directory': 'model',
 'gguf_files': ['Llama-3.2-1B-Instruct.Q8_0.gguf'],
 'modelfile_location': '/home/pdesj/work/github/ner-llm-finetuning/src/Modelfile',
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': False}

In [22]:
model.save_pretrained_gguf("model_q4_k_m", tokenizer,quantization_method="q4_k_m")

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /home/pdesj/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:30<00:00, 30.69s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:05<00:00,  5.33s/it]


Unsloth: Merge process complete. Saved to `/home/pdesj/work/github/ner-llm-finetuning/src/model_q4_k_m`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['Llama-3.2-1B-Instruct.BF16.gguf']
Unsloth: [2] Converting GGUF bf16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['Llama-3.2-1B-Instruct.Q4_K_M.gguf']
Unsloth: example usage for text onl

{'save_directory': 'model_q4_k_m',
 'gguf_files': ['Llama-3.2-1B-Instruct.Q4_K_M.gguf'],
 'modelfile_location': '/home/pdesj/work/github/ner-llm-finetuning/src/Modelfile',
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': False}

In [24]:
!ollama create llama3_1b_it_ner_instruct:q4_k_m -f /home/pdesj/work/github/ner-llm-finetuning/src/Modelfile

gathering model components ⠙ gathering model components ⠙ gathering model components ⠸ gathering model components ⠸ gathering model components 
copying file sha256:52bc94b8a29fb7a80b14108a82cd5b843f3aaff1b6b40d4f7e25ae440de4d8b1 0% ⠋ gathering model components 
copying file sha256:52bc94b8a29fb7a80b14108a82cd5b843f3aaff1b6b40d4f7e25ae440de4d8b1 6% ⠙ gathering model components 
copying file sha256:52bc94b8a29fb7a80b14108a82cd5b843f3aaff1b6b40d4f7e25ae440de4d8b1 16% ⠹ gathering model components 
copying file sha256:52bc94b8a29fb7a80b14108a82cd5b843f3aaff1b6b40d4f7e25ae440de4d8b1 27% ⠸ gathering model components 
copying file sha256:52bc94b8a29fb7a80b14108a82cd5b843f3aaff1b6b40d4f7e25ae440de4d8b1 32% ⠼ gathering model components 
copying file sha256:52bc94b8a29fb7a80b14108a82cd5b843f3aaff1b6b40d4f7e25ae440de4d8b1 44% ⠴ gathering model components 
copying file sha256:52bc94b8a29fb7a80b14108a82cd5b843f3aaff1b6b40d4f7e25ae440de4d8b1 54% ⠦ gathering model components 
copying file sha256:52bc9

# Inference

In [21]:
!curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8," } \
    ] \
    }'

{"model":"unsloth_model","created_at":"2025-12-21T15:52:04.808218265Z","message":{"role":"assistant","content":"{\""},"done":false}
{"model":"unsloth_model","created_at":"2025-12-21T15:52:04.816172235Z","message":{"role":"assistant","content":"team"},"done":false}
{"model":"unsloth_model","created_at":"2025-12-21T15:52:04.822139203Z","message":{"role":"assistant","content":"_names"},"done":false}
{"model":"unsloth_model","created_at":"2025-12-21T15:52:04.828753002Z","message":{"role":"assistant","content":"\":[\""},"done":false}
{"model":"unsloth_model","created_at":"2025-12-21T15:52:04.834663584Z","message":{"role":"assistant","content":"C"},"done":false}
{"model":"unsloth_model","created_at":"2025-12-21T15:52:04.840950542Z","message":{"role":"assistant","content":"elt"},"done":false}
{"model":"unsloth_model","created_at":"2025-12-21T15:52:04.84754239Z","message":{"role":"assistant","content":"ic"},"done":false}
{"model":"unsloth_model","created_at":"2025-12-21T15:52:04.853630014Z","m

In [14]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Must add for generation
    return_tensors="pt",
).to("cuda")

outputs = model.generate(input_ids=inputs, max_new_tokens=64, use_cache=True, temperature=1.5, min_p=0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{"team_names":["Philadelphia 76ers","Toronto Raptors","Boston Celtics","New York Knicks","Golden State Warriors"],"player_names":["Joel Embiid","Sexton Free","Kerris Stribling","Kyle Kuzma","Nikola Vučić"]},<|eot_id|>']